In [3]:
from transformers import pipeline


generator = pipeline(task="text-generation", model="gpt2")
outputs = generator(
    text_inputs="잠뜰 TV는",
    max_length=20,
    num_return_sequences=3,
    pad_token_id=generator.tokenizer.eos_token_id
)
print(outputs)

c:\ProgramData\anaconda3\envs\TEXT_017_220_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\mathn\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mathn\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate d

[{'generated_text': '잠뜰 TV는 여브뜼�'}, {'generated_text': '잠뜰 TV는소나리�'}, {'generated_text': '잠뜰 TV는 놉나법�'}]


In [2]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/137.6 kB ? eta -:--:--
     -- ------------------------------------- 10.2/137.6 kB ? eta -:--:--
     -------- ---------------------------- 30.7/137.6 kB 435.7 kB/s eta 0:00:01
     ------------------------------------ - 133.1/137.6 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 137.6/137.6 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.4/9.0 MB 8.9 MB/s eta 0:00:01
   --- ------------------------------------ 0.8/9.0 MB 8.4 MB/s eta 0:00:01
   ----- ---------------------------------- 1.2/9.0 MB 8.6 MB/s eta 0:00:01
   ------- -------------------------------- 1.6/9.0 MB 8.8 MB/s eta 0:00:01
   --------- ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
# Comment 데이터를 갖고와서 학습시킨 다음에, 키워드를 주면 그에 대한 답변을 생성하는 모델을 만들어보자
# 1. 데이터를 갖고온다
# 2. 데이터를 학습시킨다
# 3. 모델을 생성한다
# 4. 키워드를 주면 답변을 생성한다
# 5. 모델을 저장한다
# 6. 모델을 불러온다
# 7. 키워드를 주면 답변을 생성한다

import pandas as pd
df = pd.read_csv('../Data/Comment.csv')
# df.comment에 댓글들 70,000개가 있다


In [13]:
import re
def extract_korean(text):
    return re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', text)


df['comment'] = df['comment'].apply(extract_korean)
df

,comment,author,date,num_likes
0,저주받은검도계승한적있습니다거의무기콜렉터ㅋㅋㅋ,@sleepground,2024-04-18T06:52:03Z,513
1,ㅋㅋㅋㅋㅋㅋ,@yk_0720,2024-04-18T06:53:32Z,2
2,ㅋㅋㅋ,@user-me5bf3gy2v,2024-04-18T06:53:37Z,2
3,무기컬렉터ㅋㅋㅋㅋ,@ShabetOo0,2024-04-18T06:54:09Z,4
4,혹시저도계승할수있나요,@user-vr3zh8co1f,2024-04-18T06:54:13Z,3
...,...,...,...,...
73738,제가이영상에왔을때이영상이초전에올라왔다고떠있었어욤,@Doexn_,2024-01-28T07:25:21Z,0
73739,와대박이게바로이세카이,@woohyukjang3473,2024-01-28T07:14:58Z,0
73740,오늘도영상잘볼게요,@ohoeui,2024-01-28T07:14:57Z,0
73741,오,@user-vg9tx3wn4d,2024-01-28T07:14:57Z,0


In [2]:
from konlpy.tag import Okt
from collections import Counter

# 데이터 전처리
def preprocess_comments(comments):
    okt = Okt()
    tokenized_comments = [okt.morphs(comment) for comment in comments]
    return tokenized_comments

# 단어 사전 만들기
def create_vocab(tokenized_comments, min_freq=2):
    vocab = Counter()
    for comment in tokenized_comments:
        vocab.update(comment)
    vocab = {word for word, freq in vocab.items() if freq >= min_freq}
    word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}
    word_to_idx['<PAD>'] = 0
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}
    return word_to_idx, idx_to_word

# 전처리된 댓글
tokenized_comments = preprocess_comments(df['comment'])

# 단어 사전 생성
word_to_idx, idx_to_word = create_vocab(tokenized_comments)

# 단어 사전 크기 확인
vocab_size = len(word_to_idx)
print("단어 사전 크기:", vocab_size)


KeyboardInterrupt: 

In [21]:
df['comment'].to_list()

['저주받은검도계승한적있습니다거의무기콜렉터ㅋㅋㅋ',
 'ㅋㅋㅋㅋㅋㅋ',
 'ㅋㅋㅋ',
 '무기컬렉터ㅋㅋㅋㅋ',
 '혹시저도계승할수있나요',
 '헉다음시리즈도기대할게요',
 '삐빅주인은잠뜰님으로화인했습니다',
 '아니저번엔저주받은검계승하시더니이번엔전설의활을계승하시네요',
 '덕개님반응하시는거ㅋㅋㅋㅋㅋ',
 '활활타는활연구소에서스켈레톤에게활을빼앗겨활활타는활을들고있는스켈레톤에게활활타는화살을활활타는사막에서맞아준다',
 '사랑해요',
 '이건연구원이아니라용사아닌가요ㅋㅋㅋㅋㅋ',
 '직업체험을진짜다녀왔',
 '전설의유튜브의주인을계승중입니다',
 '프로필완전예뻐요',
 '영상에사용된모드의이름은뭔가요',
 '',
 '어딘가익숙한활활타오르는활연구소장ㅋㅋㅋㅋ',
 '전뜰님에훌륭한활이되고야말겟습니다',
 '',
 'ㅋ',
 '기억하시는분이안계신데요',
 '미궁굿즈왔어요',
 '스포방지선',
 '휘바휘바슈바슈바',
 '머시따',
 '수현님이초반에너무불상해',
 '재미를계승중입니다',
 '멋진영상멋진탤런트멋진디테일최강의영상이다',
 '요즘컨텐츠다들진심으로즐기면서하는거같아서더재밌는거같아요좀편안해졌달깤ㅋㅋ재밌당ㅋㅋㅋ',
 '카리스마안갈수가없겠어요ㅎㅎ',
 '후훗여기중있나후훗',
 '그런거한적없지',
 '너무재밌어서한참을웃고가네요ㅎㅎ언제나재밌는영상올려주셔서감사합니다',
 '다행스럽게도검처럼일반활돼는결말아니여서다행이네예전에검도나오자마자봐서스토라더알고있어요',
 '잠뜰님의유튜브는제가계승받겠습니다',
 '꽁꽁얼어붙은한강위로박잠뜰이걸어다닙니다',
 '오늘썸네일개레전드츄베릅',
 '요즘조회수가평균보다조금낮은이유시험시간이라서근데나는보고있다ㅋ',
 '잠뜰님그래서연구소는어떻게되나요',
 '어떤사과가한짓과똑같군요',
 '활연구소가활활타오른다휘바휘바하면슈바슈바된다ㅋㅋㅋ오늘도쩌는드립력',
 '악뜰님이랑덕개님넘기여워요ㅠㅠㅠㅠ',
 'ㅋㅋ너무재밌어요ㅋㅋㅋㅋ',
 'ㅋㅋ너무재밌어요ㅋㅋㅋㅋ',
 '',
 '다음은저주받은창인가',
 '마인크래프트에새로나온늪지스켈레톤이좀더강하게추가되었네요오늘도힐링하고가요',
 '전설의

In [13]:
from transformers import TextDataset, DataCollatorForLanguageModeling
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import TensorDataset



tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# 텍스트 데이터셋 생성
text_data = ["Your text data here...", "More text data...", "Even more text..."]
dataset = TextDataset(tokenizer=tokenizer, file_path=None, block_size=128, overwrite_cache=False)

# 언어 모델링을 위한 데이터 콜레이터 생성
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer 객체 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# 학습 실행
trainer.train()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
C:\Users\mathn\AppData\Roaming\Python\Python38\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [22]:
df=pd.read_csv('../Data/Comment.csv')
df.comment.to_csv("test.csv", index=False)

In [24]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# 텍스트 데이터셋 생성

dataset = TextDataset(tokenizer=tokenizer, file_path="./test.txt", block_size=128, overwrite_cache=False)

# 언어 모델링을 위한 데이터 콜레이터 생성
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer 객체 생성 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# 학습 실행
trainer.train()



  0%|          | 0/918 [00:00<?, ?it/s]

In [20]:
# 시작 문장
prompt_text = "today is"

# 텍스트를 토큰 ID의 시퀀스로 변환
input_ids = tokenizer.encode(prompt_text, return_tensors="pt")

# 텍스트 생성
output = model.generate(
    input_ids=input_ids,
    max_length=100,  # 생성할 최대 길이 설정
    num_return_sequences=1,  # 생성할 텍스트 수 설정
    temperature=0.7,  # 샘플링 온도 설정 (낮을수록 보수적인 결과, 높을수록 더 다양한 결과)
    top_k=50,  # 상위 k개의 토큰 중에서만 샘플링
    top_p=0.9,  # 누적 확률이 이 값을 넘지 않도록 하여 상위 p%의 토큰만 사용
    pad_token_id=tokenizer.pad_token_id,  # 패딩 토큰 ID
    eos_token_id=tokenizer.eos_token_id,  # 문장 종료 토큰 ID
    bos_token_id=tokenizer.bos_token_id,  # 문장 시작 토큰 ID
    num_beams=1,  # 빔 서치 사용 (다양한 결과를 얻기 위해 빔 서치를 사용할 수도 있음)
    early_stopping=True,  # 조건에 만족하는 첫 번째 생성 결과 반환
)

# 생성된 텍스트 출력
for i, sequence in enumerate(output):
    print(f"Generated sequence {i+1}: {tokenizer.decode(sequence, skip_special_tokens=True)}\n")


Generated sequence 1: today is not a secondary time. There is not a secondary time. There is not a secondary time. There is not a secondary time. There is not a secondary time. There is not a secondary time. There is not a sec



In [11]:
for i in dataset:
    print(i)
    break

(tensor([44204, 10600,  9606,  7235, 22376,  9211,  9019, 16913,  7182,   403,
          440,   456,   405, 36078, 13830, 11707,  7408,  8539,   739,   605,
          605,   605,   403,   440,   456,   405,     5,   403,   439, 18125,
        10435,   444,   404,   377, 15970,   458, 26987,   401, 29139, 43804,
          389,   463, 44682, 17766,   443,   389,   441, 11308,   390,   461,
        10272, 10707,   406,   460,   404,   412,   421,   419,   409,   400,
          450,   455,   418,   437,   422,   449,   377,   405, 15970,   458,
        26987,   401, 29139,   463, 44682,   459,   389,   440,   443,   390,
          412,   421,   419,   409,   400,   450,   455,   418,   437,   422,
          449,   403,   390,   439,   405,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3]

In [12]:
trainer.train()

  0%|          | 0/27654 [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'keys'

In [4]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   -------- ------------------------------ 30.7/139.4 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 139.4/139.4 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/215.0 kB ? eta -:--:--
   --------------------------------------- 215.0/215.0 kB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------  2.3/2.3 MB 48.6 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 36.9 MB/s eta 0:00:00


In [18]:
!go install github.com/slsa-framework/slsa-verifier/v2/cli/slsa-verifier@v2.5.1

'go'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
